In [1]:
import os
import json
import joblib
import time

import pandas as pd
import random
from IPython.display import display, HTML

In [2]:
import torch
import transformers
from transformers import BartTokenizer, BartForConditionalGeneration

In [3]:
#connect to hugging face
!git clone https://github.com/huggingface/transformers \
&& cd transformers \
install
!pip install -q ./transformers

fatal: destination path 'transformers' already exists and is not an empty directory.


In [4]:
# Installing transformers
!pip install simpletransformers
!pip install transformers
# Importing necessary libraries from simple transformers
import pandas as pd
from simpletransformers.seq2seq import Seq2SeqModel,Seq2SeqArgs

In [5]:
pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Using: ', device)

Using:  cuda


In [7]:
headlines_processed = []
clean_news = []
dataset = []
target = []
count = 0

In [8]:
path_models = './models/'
path_outputs = './outputs/'
path_processed_data= './processed_data/'

In [9]:
jsonFile = open(os.path.join(path_processed_data, 'cnn_train.json'), "r")
jsonContent = jsonFile.read()
details_dict = json.loads(jsonContent)
jsonFile.close()

X_train = details_dict['text']
Y_train = details_dict['headlines']

jsonFile = open(os.path.join(path_processed_data, 'cnn_valid.json'), "r")
jsonContent = jsonFile.read()
details_dict = json.loads(jsonContent)
jsonFile.close()

X_valid = details_dict['text']
Y_valid = details_dict['headlines']

jsonFile = open(os.path.join(path_processed_data, 'cnn_test.json'), "r")
jsonContent = jsonFile.read()
details_dict = json.loads(jsonContent)
jsonFile.close()

X_test = details_dict['text']
Y_test = details_dict['headlines']


In [10]:
print(len(X_train), len(Y_train), len(X_valid), len(Y_valid), len(X_test), len(Y_test))

73862 73862 9233 9233 9233 9233


In [11]:
X_train[1:2]

['JERUSALEM  would bring catastrophe .  You need rabbi this .  The Israeli Prisons Authority said unaware Amir done interviews require prior departmental permission .  As result Amir moved highersecurity prison phone calls family visits conjugal visits revoked according Prisons Authority spokesman .  On November 4 1995 Amir shot Rabin Rabin leaving rally Tel Aviv support Oslo Accords set plan permanent peace Israel Palestinians .  All About Yitzak Rabin  Israel']

In [12]:
Y_train[1:2]

['Fury Rabin killer TV interviews']

In [13]:
details = {
    'headlines' : Y_train,
    'text' : X_train
}

df_train = pd.DataFrame(details)

In [14]:
details = {
    'headlines' : Y_valid,
    'text' : X_valid
}

df_valid = pd.DataFrame(details)

In [15]:
details = {
    'headlines' : Y_test,
    'text' : X_test
}

df_test = pd.DataFrame(details)

In [30]:
# Import


tokenizer = BartTokenizer.from_pretrained("facebook/bart-large")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large")

torch_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Using: ', torch_device)


Using:  cuda


In [31]:
# Rename columns as per pretrained model required format
df_train=df_train.rename(columns={'headlines':'target_text','text':'input_text'})
df_valid=df_valid.rename(columns={'headlines':'target_text','text':'input_text'})
df_test=df_test.rename(columns={'headlines':'target_text','text':'input_text'})
#df = {dataset, target}
model_args = Seq2SeqArgs()
# Initializing number of epochs
#model_args.num_train_epochs = 25
model_args.num_train_epochs = 2
# Initializing no_save arg
model_args.no_save = True
model_args.overwrite_output_dir = True

# Initializing evaluation args
model_args.evaluate_generated_text = True
model_args.evaluate_during_training = True
model_args.evaluate_during_training_verbose = True

In [32]:
df_train['target_text'][0]

'The promise World War I could keep'

In [33]:
df_train['input_text'][0]

'Editor note This eighth series legacies World War I appearing CNN . comOpinion weeks leading 100year anniversary war outbreak .  Ruth BenGhiat guest editor series .  Samuel Moyn professor law history Harvard University .  His new book Human Rights Uses History .   America recent misdeeds minor .  One big reason U . S .  military response Vietnam After decades benign neglect outright disregard started treat laws war real constraints fights .  Opinion When chemical weapons killed 90000 Consider decision target enemies make sure combatants rather civilians crosshairs Vietnam like World War I lawyers nowhere found civilians dying massive numbers .  Today military government lawyers play central role recent controversy legal authorization drone strike Anwar alAwlaki attests .  The long trajectory World War I produced paradoxical situation We real though insufficient constraints war brutality public discourse ever it .  But lack much concern goal eliminating war itself .  One legacy horrifi

In [34]:
df_valid['target_text'][0]

'Chinese moon rover launched successfully'

In [35]:
df_valid['input_text'][0]

'Story highlights China unmanned space craft successfully released moon rover Three nations softlanded moon surface The Jade Rabbit rover study lunar surface China rapidly built space program China first lunar rover deployed successfully unmanned spacecraft Change3 landed moon Saturday .  Jade Rabbit  surface less five days later .  On Tuesday descended elliptical orbit lowest point 15 kilometers lunar surface spokesperson China Administration Science Technology Industry National Defense told Xinhua .  The Soviet Union Luna 24 probe last space mission land moon August 1976  four years United States launched manned Apollo 17 mission .  Iran claims second launch monkey space back'

In [36]:
df_test['target_text'][0]

'Germanwings crash report Pilot locked cockpit'

In [37]:
df_test['input_text'][0]

'Story highlights Lufthansa We information corroborate New York Times report Report Audio recording includes sounds pilot trying smash cockpit door Teams still searching flight data recorder  Latest developments  One pilots board Germanwings Flight 9525 locked cockpit plane crashed senior military official told The New York Times Wednesday citing evidence cockpit voice recorder .   Lufthansa parent company Germanwings said we information bodies investigating incident would corroborate report The New York Times .  We participate speculation follow matter .   Helicopters airlifted victims remains site Germanwings plane crash French Alps Gendarmerie said Wednesday according CNN affiliate France 2 .  Full story Investigators trying determine caused crash Germanwings Flight 9525 made startling discovery audio recording according New York Times report One plane pilots locked cockpit crash .  You hear trying smash door down senior military official involved investigation told newspaper descri

In [38]:
model = Seq2SeqModel(
encoder_decoder_type="bart",
encoder_decoder_name="facebook/bart-large",
args=model_args,
use_cuda=True,
)
# Splitting data into train-test
# from sklearn.model_selection import train_test_split
# train_df, test_df = train_test_split(df, test_size=0.2)


In [39]:
df_train.shape, df_valid.shape, df_test.shape

((73862, 2), (9233, 2), (9233, 2))

In [40]:
len(df_train)//10000

7

In [41]:
def current_time_min():
    time_in_mins = time.time()/60.0
    return round(time_in_mins, 4)

In [ ]:
# Training the model and keeping eval dataset as test data
# 16000 for aws, 800 for local VM
batch_size_train = 10000
batch_size_test = batch_size_train//4
number_of_batches = len(df_train)//batch_size_train
#count = 0
epocs = model_args.num_train_epochs
#acc_time = 0

for i in range(number_of_batches):
    start_time = current_time_min()
    count += 1
    start_offset_train = i * batch_size_train
    start_offset_test = i * batch_size_test
    end_offset_train = start_offset_train + batch_size_train
    end_offset_test = start_offset_test + batch_size_test
    model.train_model(df_train[start_offset_train:end_offset_train], eval_data=df_valid)
    model_filename = "model_" + str(count) + "_" + str(start_offset_train) + "_" + str(end_offset_train) + "_" + str(epocs) + "e.pkl"
    joblib.dump(model, os.path.join(path_models, model_filename))
    results = model.eval_model(df_test)
    batch_time = current_time_min() - start_time
    acc_time = acc_time + batch_time/60
    randint = random.randrange(1, len(df_test), 3)
    results_json = {
        'iteration' : count,
        'start' : start_offset_train,
        'end' : end_offset_train,
        'eval_loss' : results["eval_loss"],
        'batch_time' : batch_time,
        'acc_time' : acc_time
    }
    jsonString = json.dumps(results_json)
    results_filename = "results_" + str(start_offset_train) + "_" + str(end_offset_train) + "_" + str(count)
    results_file = open(os.path.join(path_models, "results.txt"), "a")
    results_file.write(jsonString + '\n')
    results_file.close()
    print("Count: ", count, "Results: ", results)


  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/1250 [00:00<?, ?it/s]

  0%|          | 0/9233 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/1155 [00:00<?, ?it/s]

...

  0%|          | 0/9233 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/1155 [00:00<?, ?it/s]

  0%|          | 0/9233 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1155 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/1155 [00:00<?, ?it/s]

In [ ]:
# Generating summaries on news test data
results = model.eval_model(df_test[100:200])
# print the loss
results

In [ ]:


# MODEL_PATH = os.path.join(path_models, 'bart.model')

# torch.save(model, MODEL_PATH)

In [ ]:
joblib.dump(model, os.path.join(path_models, 'model.pkl'))

In [ ]:
model = Seq2SeqModel(encoder_decoder_type="bart",
  encoder_decoder_name="facebook/bart-large",
  args=model_args,
  use_cuda=False,)

model.load(torch.load(MODEL_PATH))


In [ ]:
model = joblib.load(os.path.join(path_models, 'model.pkl'))

In [ ]:
# model.eval()

In [ ]:
# Original test data text summary for top 10 news
#value = randint(0, 10)
for i in test_df.target_text[422:423]:
  print(i)



In [ ]:
# Predicted summary
for i in test_df.input_text[1000:1050]:
  print(model.predict([i]))